## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
data_file_location = '/content/drive/My Drive/SYSC 4907 COVID & Deep Learning/Model Training/'
training_data_file = 'FINAL_COMBINED_3_TRAIN.xlsx'
training_data_path = data_file_location + training_data_file
training_data = pd.read_excel(training_data_path)
training_data.head(10)

,DATA,LABEL,Unnamed: 2,Unnamed: 3
0,While the labor market is showing signs of rec...,0,"03/07/2021, 17:52:15",1368620525124079616
1,Important update—Vermonters ages 70 and up are...,1,"02/18/2021, 18:42:16",1362472519459078144
2,COVID-19 as it happens #EU #EuropeanUnion #Eur...,2,"02/24/2021, 19:46:09",1364662922581069826
3,We want to hear from YOU! We’re holding our CO...,2,"02/18/2021, 18:41:01",1362472203225284608
4,THE•ULTIMATE•FIRST•YEAR•EXPERIENCE is official...,2,"02/18/2021, 18:42:02",1362472459132411915
5,TVToronto There should be a tax reduction to h...,1,"02/18/2021, 18:45:17",1362473278263197698
6,I challenged Boris Johnson in Parliament this ...,0,"02/24/2021, 19:38:04",1364660886212927492
7,Slavitt helped spark panic around ventilator a...,0,"03/07/2021, 22:21:36",1368688310642675720
8,(1) Women in power can espouse internalised mi...,1,"02/18/2021, 18:45:15",1362473270348369922
9,"Due to pandemic, small businesses are having a...",2,"03/07/2021, 18:41:36",1368632944978325506


In [ ]:
test_data_file = 'FINAL_COMBINED_3_TEST.xlsx'
test_data_path = data_file_location + test_data_file
test_data = pd.read_excel(test_data_path)
test_data.head(10)

,DATA,LABEL,Unnamed: 2,Unnamed: 3
0,Why should anyone have to hack for this techno...,1,"02/18/2021, 18:40:36",1362472099185590273
1,Did they test positive for COVID?,1,"03/07/2021, 17:52:06",1368620488063324166
2,Our EY COVID-19 Response Group supporting #Thu...,2,"02/18/2021, 18:45:07",1362473236781498370
3,Joe Biden's Coronavirus Coordinator just said ...,0,"03/07/2021, 17:52:30",1368620587027927042
4,#Tanzania: Vice President uluhuSamia with no f...,1,"02/18/2021, 18:45:07",1362473237414821890
5,"COVID-19 hospitalizations dropped to 1,137 Thu...",1,"02/18/2021, 18:41:21",1362472286624751618
6,Look like it nuh sink ina your cerebrum yet......,0,"03/07/2021, 17:52:26",1368620569386758149
7,"*spring 2023* PRINCETON, N.J. — The Ivy League...",1,"02/18/2021, 18:45:05",1362473227163996164
8,"It’s not just the industry trifecta"" of chicke...",0,"02/24/2021, 19:36:47",1364660566824992769
9,"Fuck Johnathan Corona, all the homies hate Joh...",0,"02/18/2021, 18:45:25",1362473312505356289


In [ ]:
training_data.LABEL.value_counts()

2    895
0    895
1    867
Name: LABEL, dtype: int64

In [ ]:
test_data.LABEL.value_counts()

0    229
1    208
2     76
Name: LABEL, dtype: int64

In [ ]:
train_subset = training_data[['DATA', 'LABEL']]
x_train = train_subset['DATA'].to_numpy()
y_train = train_subset['LABEL'].to_numpy()
test_subset = test_data[['DATA', 'LABEL']]
x_val = test_subset['DATA'].to_numpy()
y_val = test_subset['LABEL'].to_numpy()
print(x_train[0:5])
print(len(x_train), "Training sequences")
print(y_train[0:5])
print(len(x_val), "Testing sequences")

["While the labor market is showing signs of recovery, millions who lost jobs at the start of the pandemic are still out of work. Those who are long-term unemployed accounted for 41% of all unemployed people ' levels not seen since the Great Recession."
 'Important update—Vermonters ages 70 and up are now eligible to make an appointment for FREE COVID-19 vaccines!'
 'COVID-19 as it happens #EU #EuropeanUnion #Europe #UnitedKingdom #UK #GreatBritain #GB #COVID19 #Healthcare #Leadership #pandemic #vaccine #economy #Infographic #Infographics via rJDrooghaag'
 'We want to hear from YOU! We’re holding our COVID-19 Town Hall for African Americans Monday, February 22 at 6:30 p.m. During the Town Hall, you’ll hear from medical experts, community leaders, and public health professionals. Comment below with your questions on the vaccine!'
 'THE•ULTIMATE•FIRST•YEAR•EXPERIENCE is officially open for Registration #UA25 and #BamaTransfers - Register TODAY at to secure your spot! Tag a friend below t

## Implement a Transformer block as a layer

## Implement embedding layer

Two seperate embedding layers, one for tokens, one for token index (positions).

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
!pip install -q transformers
from transformers import TFBertForSequenceClassification, BertTokenizer

module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
tokenizer = BertTokenizer(vocab_file, do_lower_case=True)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len

        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)

      return np.array(all_tokens), np.array(all_masks), np.array(all_segments)
      
    

In [ ]:
max_len=40
train_input = bert_encode(training_data.DATA.values, tokenizer, max_len=max_len)
test_input = bert_encode(test_data.DATA.values, tokenizer, max_len=max_len)
train_labels = keras.utils.to_categorical(training_data.LABEL.values, num_classes=3)
test_labels = keras.utils.to_categorical(test_data.LABEL.values, num_classes=3)

In [ ]:
x_val[0:10]

array(["Why should anyone have to hack for this technology why isn't it just free to everyone How does anyone think this makes North Korea look like the bad guys",
       'Did they test positive for COVID?',
       'Our EY COVID-19 Response Group supporting #ThumbsUpForEarlyYears',
       "Joe Biden's Coronavirus Coordinator just said on National television that ' people of color ' are being confused by too many websites that dont work & how to access computers. Why are so many racist statements, words & comments coming directly from the White House under Biden?",
       '#Tanzania: Vice President uluhuSamia with no face mask as she attended burial ceremony of Zanzibar 1st VP, Maalim Seif Sharif Hamad, who died of COVID-19 yesterday.',
       'COVID-19 hospitalizations dropped to 1,137 Thursday, their lowest point in more than two months and about 54% lower than their peak of 2,466 just over a month ago.',
       'Look like it nuh sink ina your cerebrum yet.... Action/words',
       '*

In [ ]:
test_input[0:10]

(array([[ 101, 2339, 2323, ...,    0,    0,    0],
        [ 101, 2106, 2027, ...,    0,    0,    0],
        [ 101, 2256, 1041, ...,    0,    0,    0],
        ...,
        [ 101, 1045, 1521, ..., 7484, 2824,  102],
        [ 101, 5364, 2231, ...,    0,    0,    0],
        [ 101, 4701, 4084, ...,    0,    0,    0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [ ]:
test_labels[0:10]

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [ ]:
labels_train_y = tf.one_hot(y_train, 3).numpy()
labels_test_y = tf.one_hot(y_val, 3).numpy()

In [ ]:
train_data = train_subset

In [ ]:
y_train

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
labels_train_y[7]

2

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(5, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    
    return model

In [ ]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 40)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 40)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 40)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [ ]:
train_history = model.fit(x=train_input, y=train_labels, epochs=10, batch_size=32, validation_data=(test_input, test_labels))

Epoch 1/10


ValueError: ignored

In [ ]:
data = test_input
result = model.predict(data[0][0], data[1][0], data[2][0])

class_name=('negative','neutral','positive')
r_list = []
c_list = []
for r in result:
  max = np.argmax(r)
  r_list.append(max)
  c_list.append(class_name[max])
print(r_list) #the prdicted index of the class, in this case is 0 1 2 it migth need for visulization
print(c_list)#actual result

ValueError: ignored

In [ ]:
print(data[0][0])

[  101  2339  2323  3087  2031  2000 20578  2005  2023  2974  2339  3475
  1005  1056  2009  2074  2489  2000  3071  2129  2515  3087  2228  2023
  3084  2167  4420  2298  2066  1996  2919  4364   102     0     0     0
     0     0     0     0]


In [ ]:
model_name = 'bert-base-uncased'

max_length = 40

config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

transformer_model  = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=3, output_attentions=False, output_hidden_states=False)

NameError: ignored

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = {'LABEL': CategoricalCrossentropy(from_logits = True)}
metric = {'LABEL': CategoricalAccuracy('accuracy')}

transformer_model.compile(optimizer, loss=loss, metrics=metric)

In [ ]:
learning_rate = 1e-5

# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3, output_attentions=False, output_hidden_states=False)

# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

#model.compile(optimizer, loss='sparse_categorical_crossentropy', metrics=["sparse_categorical_accuracy"])
model.compile(optimizer, loss='categorical_crossentropy', metrics=["categorical_accuracy"])

INFO:filelock:Lock 139959299495696 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170.lock


INFO:filelock:Lock 139959299495696 released on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170.lock


INFO:filelock:Lock 139959306074064 acquired on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock


INFO:filelock:Lock 139959306074064 released on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
